In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, TrainerCallback
from trl import AutoModelForCausalLMWithValueHead, DPOTrainer, DPOConfig
from huggingface_hub import login

from openai import OpenAI
from difflib import SequenceMatcher

from datasets import Dataset
import pandas as pd

import os
from pathlib import Path
from dotenv import load_dotenv

import DPO_pairwise_human as dp

load_dotenv()
PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT")).resolve()
MODEL_ROOT = Path(os.getenv("MODEL_ROOT")).resolve()
DATA_ROOT = Path(os.getenv("DATA_ROOT")).expanduser().resolve()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# show the GPU device and number of GPUs available
print(f"Using device: {device}")
print(f"Number of GPUs available: {torch.cuda.device_count()}")

In [ ]:
print("loading model and tokenizer...")
model_path = MODEL_ROOT / 'test' / 'TinyLlama-1.1B-Chat-v1.0'

tokenizer = AutoTokenizer.from_pretrained(str(model_path))
model = AutoModelForCausalLMWithValueHead.from_pretrained(str(model_path)).to(device)
model.warnings_issued = {}
print("model loaded")

In [ ]:
print('loding evaluation dataset...')
dataset_path = DATA_ROOT / '.kaggle' / 'cnn_dailymail' / 'validation.csv'
df = pd.read_csv(dataset_path, nrows=1)
df = df[['article', 'highlights']].rename(columns={
    'article': 'content',
    'highlights': 'answer'
})
dataset = Dataset.from_pandas(df)
print('dataset loaded')

In [ ]:
raw_data = dp.prepare_raw_data(model, tokenizer, dataset)

pairwise_data = dp.prepare_pairwise_data(raw_data)

In [ ]:
score = dp.compute_win_rate(model, tokenizer, pairwise_data)

In [ ]:
print(score)

In [ ]:
print(raw_data[0])
print(pairwise_data[0])